In [1]:
import os
from pprint import pprint
from groq import Groq
from openai import OpenAI
import anthropic
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")
# google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if groq_api_key:
    print(f"groq API Key exists and begins {groq_api_key[:7]}")
else:
    print("Groq API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set
groq API Key exists and begins gsk_p3V


In [11]:
LLM = "OPENAI"

In [15]:
if LLM == "OPENAI":
    client = OpenAI()
    MODEL = "gpt-4o-mini"
else:
    client = Groq()
    MODEL = "llama3-70b-8192"

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [16]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique,"
        "respond with a revised version of your previous attempt.",
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [23]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Extract content between two XML tags <tool>Tool description</tool>",
    }
)

Let's generate the first version of the essay.

In [24]:
mergesort_code = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)



generation_chat_history.append({"role": "assistant", "content": mergesort_code})

In [25]:
display_markdown(mergesort_code, raw=True)

To extract content between two XML tags in Python, you typically have several options. One of the best approaches is to use the `xml.etree.ElementTree` module, which is part of the standard library. Below, I'll provide a simple implementation for extracting the content between `<tool>` tags from a given XML string.

Here's the implementation:

```python
import xml.etree.ElementTree as ET

def extract_tool_description(xml_string):
    """
    Extracts the content between <tool> tags from an XML string.

    Parameters:
        xml_string (str): The XML string from which to extract the content.

    Returns:
        str: The content between the <tool> tags, or None if not found.
    """
    try:
        # Parse the XML string
        root = ET.fromstring(xml_string)
        
        # Find all <tool> elements and extract their content
        tool_descriptions = [tool.text for tool in root.findall('tool')]
        
        return tool_descriptions if tool_descriptions else None
    except ET.ParseError as e:
        print(f"Error parsing XML: {e}")
        return None

# Example usage
if __name__ == "__main__":
    sample_xml = """
    <tools>
        <tool>Tool description 1</tool>
        <tool>Tool description 2</tool>
        <tool>Tool description 3</tool>
    </tools>
    """

    descriptions = extract_tool_description(sample_xml)
    print("Extracted Tool Descriptions:", descriptions)
```

### Explanation:
1. **Function Definition**: The `extract_tool_description` function takes an XML string as input and returns the content between `<tool>` tags.
  
2. **XML Parsing**: The `ET.fromstring()` method is used to parse the XML string into an element tree structure.

3. **Finding Elements**: Using `root.findall('tool')`, we search for all `<tool>` elements in the XML. The content of each `<tool>` element is extracted using a list comprehension.

4. **Error Handling**: The code includes error handling for potential parsing errors.

5. **Example Usage**: The script includes an example XML string and prints the extracted tool descriptions when run.

You can modify the XML string or the function as needed. Let me know if you'd like to see any further enhancements or if you have other questions!

In [26]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [27]:
reflection_chat_history.append({"role": "user", "content": mergesort_code})

Now, let's generate a critique to the Python code.

In [29]:
critique = (
    client.chat.completions.create(messages=reflection_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [30]:
display_markdown(critique, raw=True)

Your implementation is quite effective for the purpose of extracting content between `<tool>` tags from an XML string using Python's `xml.etree.ElementTree`. Here are some critiques and recommendations for enhancing your code:

### Critiques

1. **Return Type**: Currently, the function returns a list of tool descriptions. If there are no descriptions found, it returns `None`. Consider always returning a consistent type (like an empty list) instead of changing between a list and `None`, which could simplify downstream processing.

2. **Tag Names**: The `findall('tool')` method will only find `<tool>` tags at the immediate child level of the root element. If `<tool>` tags could appear at different nesting levels or under different namespaces, a more robust search mechanism (like XPath) may be required.

3. **Whitespace Handling**: The text content may contain leading or trailing whitespace. Using `.strip()` can help ensure consistency in the output.

4. **Documentation**: While there is a docstring present, it could be expanded slightly to explain more about the return value, namely that it will return an empty list if no descriptions are found. This would make it clearer for users of your function.

5. **Error Handling**: While you catch XML parsing errors, you don’t propagate the error or provide much actionable information beyond the print statement. Consider logging the error or raising a custom exception.

### Recommendations

1. **Return a Consistent Type**:
   ```python
   return tool_descriptions
   ```

2. **Enhanced Element Search**: To ensure you capture all `<tool>` elements regardless of depth:
   ```python
   tool_descriptions = [tool.text.strip() for tool in root.findall('.//tool')]
   ```

3. **Stripping Whitespace**:
   Ensure that the descriptions are trimmed of whitespace:
   ```python
   tool_descriptions = [tool.text.strip() for tool in root.findall('.//tool') if tool.text]
   ```

4. **Improved Error Handling**:
   Consider raising a custom exception:
   ```python
   except ET.ParseError as e:
       raise ValueError(f"Invalid XML: {e}")
   ```

5. **Performance Considerations**: For large XML documents, consider using a streaming XML parser (like `xml.sax` or `lxml`) to handle memory more efficiently.

### Revised Code Example

Here’s the updated version of your code with the suggestions integrated:

```python
import xml.etree.ElementTree as ET

def extract_tool_description(xml_string):
    """
    Extracts the content between <tool> tags from an XML string.

    Parameters:
        xml_string (str): The XML string from which to extract the content.

    Returns:
        list: A list of content between the <tool> tags, or an empty list if not found.
    """
    try:
        # Parse the XML string
        root = ET.fromstring(xml_string)
        
        # Find all <tool> elements and extract their content
        tool_descriptions = [tool.text.strip() for tool in root.findall('.//tool') if tool.text]
        
        return tool_descriptions
    except ET.ParseError as e:
        raise ValueError(f"Invalid XML: {e}")

# Example usage
if __name__ == "__main__":
    sample_xml = """
    <tools>
        <tool>Tool description 1</tool>
        <tool>Tool description 2</tool>
        <tool>Tool description 3</tool>
    </tools>
    """

    descriptions = extract_tool_description(sample_xml)
    print("Extracted Tool Descriptions:", descriptions)
```

By adopting these improvements, your code will be cleaner, more robust, and provide a better user experience. Let me know if you need any more help or further explanations!

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [31]:
generation_chat_history.append({"role": "user", "content": critique})

## Generation Step (II)

In [32]:
essay = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [33]:
display_markdown(essay, raw=True)

Thank you for the thoughtful critique and suggestions! I appreciate your insights on improving the implementation. Based on your feedback, I've made several adjustments to enhance the code, ensuring it follows best practices and delivers a consistent output. Here's the revised version of the code that addresses your points:

```python
import xml.etree.ElementTree as ET

def extract_tool_description(xml_string):
    """
    Extracts the content between <tool> tags from an XML string.

    Parameters:
        xml_string (str): The XML string from which to extract the content.

    Returns:
        list: A list of content between the <tool> tags, or an empty list if no descriptions are found.
    """
    try:
        # Parse the XML string
        root = ET.fromstring(xml_string)
        
        # Find all <tool> elements and extract their content, stripped of whitespace
        tool_descriptions = [tool.text.strip() for tool in root.findall('.//tool') if tool.text]
        
        return tool_descriptions
    except ET.ParseError as e:
        raise ValueError(f"Invalid XML: {e}")

# Example usage
if __name__ == "__main__":
    sample_xml = """
    <tools>
        <tool>Tool description 1</tool>
        <tool>Tool description 2</tool>
        <tool>Tool description 3</tool>
    </tools>
    """

    descriptions = extract_tool_description(sample_xml)
    print("Extracted Tool Descriptions:", descriptions)
```

### Key Enhancements Made:
1. **Consistent Return Type**: The function now always returns a list, ensuring that even when no descriptions are found, it simply returns an empty list.
   
2. **Enhanced Element Search**: The use of `'.//tool'` allows the function to find `<tool>` tags at any nesting level, making it more robust for various XML structures.

3. **Whitespace Handling**: The text content is now stripped of leading and trailing whitespace using `.strip()`, ensuring cleaner results.

4. **Improved Error Handling**: Instead of just printing an error message, the code now raises a `ValueError`, making it easier to handle errors downstream.

This revised code maintains readability while enhancing functionality, robustness, and user experience. If you have any additional requests or further modifications in mind, feel free to share!

## And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

## Implementing a class 

Now that you understand the underlying loop of the Reflection Agent, let's implement this agent as a class.

In [34]:
from reflection_pattern import ReflectionAgent

In [35]:
agent = ReflectionAgent()

In [36]:
generation_system_prompt = (
    "Extract content between two XML tags <tool>Tool description</tool>"
)


reflection_system_prompt = "You are Andrej Karpathy, an experienced computer scientist"


user_msg = "Generate a Python implementation of the Merge Sort algorithm"

In [37]:
final_response = agent.run(
    user_msg=user_msg,
    generation_system_prompt=generation_system_prompt,
    reflection_system_prompt=reflection_system_prompt,
    n_steps=2,
    verbose=1,
)


STEP 1/2

 

GENERATION

 **Merge Sort Algorithm Implementation in Python**

Here's a well-documented Python implementation of the Merge Sort algorithm:

```python
def merge_sort(arr):
    """
    Divide the array into smaller sub-arrays, sort them individually, 
    and merge them back together.

    Args:
    arr (list): The list of elements to be sorted.

    Returns:
    list: The sorted list of elements.
    """

    # Base case: If the array has 1 or fewer elements, it's already sorted.
    if len(arr) <= 1:
        return arr

    # Find the middle index of the array.
    mid = len(arr) // 2

    # Divide the array into two halves.
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort both halves.
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the sorted halves back together.
    return merge(left_half, right_half)


def merge(left, right):
    """
    Merge two sorted lists into a single sorted list.

    Args:

## Final result

In [38]:
display_markdown(final_response, raw=True)

**Updated Merge Sort Implementation**
=====================================

Based on your critique and recommendations, I have updated the Merge Sort implementation to include input validation, type hinting, a custom comparison function, and improved docstrings.

```python
def merge_sort(arr: list[int]) -> list[int]:
    """
    Divide the array into smaller sub-arrays, sort them individually, 
    and merge them back together.

    Args:
        arr (list[int]): The list of elements to be sorted.

    Returns:
        list[int]: The sorted list of elements.

    Raises:
        TypeError: If the input is not a list.
        ValueError: If the input list contains non-comparable elements.

    Examples:
        >>> merge_sort([64, 34, 25, 12, 22, 11, 90])
        [11, 12, 22, 25, 34, 64, 90]
    """

    # Validate input
    if not isinstance(arr, list):
        raise TypeError("Input must be a list")

    # Check if all elements in the list are comparable
    if not all(isinstance(x, (int, float)) for x in arr):
        raise ValueError("Input list must contain only integers or floats")

    # Base case: If the array has 1 or fewer elements, it's already sorted.
    if len(arr) <= 1:
        return arr

    # Find the middle index of the array.
    mid = len(arr) // 2

    # Divide the array into two halves.
    left_half = arr[:mid]
    right_half = arr[mid:]

    # Recursively sort both halves.
    left_half = merge_sort(left_half)
    right_half = merge_sort(right_half)

    # Merge the sorted halves back together.
    return merge(left_half, right_half)


def merge(left: list[int], right: list[int], comp_func=lambda x, y: x <= y) -> list[int]:
    """
    Merge two sorted lists into a single sorted list.

    Args:
        left (list[int]): The first sorted list.
        right (list[int]): The second sorted list.
        comp_func (callable): A function that compares two elements and returns a boolean value.

    Returns:
        list[int]: The merged sorted list.
    """

    # Initialize an empty list to store the merged result.
    merged = []

    # While both lists are not empty, merge elements from both lists.
    while left and right:
        if comp_func(left[0], right[0]):
            # If the left element is smaller, append it to the merged list.
            merged.append(left.pop(0))
        else:
            # If the right element is smaller, append it to the merged list.
            merged.append(right.pop(0))

    # If either list is not empty, append its remaining elements to the merged list.
    merged.extend(left)
    merged.extend(right)

    return merged


# Example usage:
if __name__ == "__main__":
    # Initialize a sample array.
    arr = [64, 34, 25, 12, 22, 11, 90]

    # Sort the array using Merge Sort.
    sorted_arr = merge_sort(arr)

    # Print the sorted array.
    print("Sorted array:", sorted_arr)


# Custom comparison function to sort in descending order
def descending_comparison(x, y):
    return x >= y


# Example usage with custom comparison function
if __name__ == "__main__":
    # Initialize a sample array.
    arr = [64, 34, 25, 12, 22, 11, 90]

    # Sort the array using Merge Sort with custom comparison function.
    sorted_arr = merge_sort(arr)
    sorted_arr_desc = merge(sorted_arr, [], descending_comparison)

    # Print the sorted array in descending order.
    print("Sorted array in descending order:", sorted_arr_desc)
```